In [1]:
import pandas as pdfrom sklearn.model_selection import train_test_splitfrom sklearn.ensemble import RandomForestClassifierfrom sklearn.metrics import accuracy_score, classification_reportfrom sklearn.datasets import load_iris

In [2]:
# Load the Iris datasetiris = load_iris()iris_df = pd.DataFrame(data=iris.data, columns=iris.feature_names)iris_df['target'] = iris.target# Display the first few rows of the datasetprint(iris_df.head())

In [3]:
# Prepare the data for trainingX = iris_df.drop('target', axis=1)y = iris_df['target']X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [4]:
# Train a Random Forest Classifierrf_classifier = RandomForestClassifier()rf_classifier.fit(X_train, y_train)

In [5]:
# Predict on the test set and evaluate the modely_pred = rf_classifier.predict(X_test)accuracy = accuracy_score(y_test, y_pred)print(f'Accuracy: {accuracy}')

In [6]:
# Print classification reportprint(classification_report(y_test, y_pred))